<a href="https://colab.research.google.com/github/4dsolutions/clarusway_data_analysis/blob/main/DAwPy_S1_(Numpy_Arrays)/daily_schedule.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a><br/>
[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/4dsolutions/clarusway_data_analysis/blob/main/DAwPy_S1_%28Numpy_Arrays%29/daily_schedule.ipynb)

# Synchronizing Around the S Module

To "synchronize" is a lot like to "omni-triangulate" i.e. to well-order in space and/or time.  "Synchronized swimming" is where they don't crash into each other, the whole point being to show off tightly coreographed dance moves.

Let's start with a published page from Synergetics (R. Buckminster Fuller), showing clearly what we mean by "the S module":

<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/6335726352/in/photolist-ujipN3-vLby2U-CwvwpP-5zTRjp-5zY9gA-S4BDeE-22RBsb5-25JufgG-7jZLe2-7jZLhp-7k4Eid-7k4Ejf-7k4Em5-7mcmne-2d6EFEQ-8ryEix-8ryECF-2mT2hXj-2mT3vMb-2oFtnCt-aDSfHf-f75zUP" title="S Module"><img src="https://live.staticflickr.com/6114/6335726352_902009df40_w.jpg" width="400" height="353" alt="S Module"/></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

Corresponding to this picture, is the corresponding plane net (note that the letters match):

<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/52960966455/in/datetaken/" title="f988_13A"><img src="https://live.staticflickr.com/65535/52960966455_e8acc598fa_w.jpg" width="400" height="305" alt="f988_13A"/></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

What we're going to need are the six edges of the S module tetrahedron, for one of our tetravolume computing algorithms, the ones that eat edges for input.  To this end, let's gather all the distances in the above diagram.  

Note that the plane net contains duplicate edges.  We will make sure our labels are alphabetized internally i.e. given EG and GE, we will pick EG as the canonical representation, because E < G in the alphabet.

In [1]:
import pandas as pd
import numpy as np

import math
from math import sqrt as rt2

a  = 1

EF = a * rt2(7 - 3 * rt2(5))
EG = (a/2) * rt2(7 - 3 * rt2(5))
EH = (a/2) * (3 - rt2(5))
FG = (a/2) * rt2(3) * rt2(7 - 3 * rt2(5))
FH = (a/2) * (rt2(5) - 1)
GH = (a/2) * rt2(7 - 3 * rt2(5))

S_net = pd.DataFrame({'Edges'  : ['EF', 'EG', 'EH', 'FG', 'FH', 'GH'],
                      'Lengths': [EF, EG, EH, FG, FH, GH]})

S_net

,Edges,Lengths
0,EF,0.540182
1,EG,0.270091
2,EH,0.381966
3,FG,0.467811
4,FH,0.618034
5,GH,0.270091


Now we want a volume formula that eats edge lengths in a particular order, and returns volume.

We have at least three to choose from:

* Piero della Francesca's
* Gerald de Jong's
* Cayley-Menger Determinant

All three are described [here](https://nbviewer.org/github/4dsolutions/School_of_Tomorrow/blob/master/VolumeTalk.ipynb).

We start with an apex vertex (any vertex), say E, extending to EF, EG, EH, and then we go around the opposite face in the same order i.e. FG GH FH.

Here we go then:

In [2]:
def CM_ivm(a, b, c, d, e, f):
    # double and 2nd power us
    A,B,C,D,E,F = [(2*x)**2 for x in (a,b,c,d,e,f)]
    
    # Construct a 5x5 matrix per Caley-Menger
    M = np.ones((5,5))
    M[0,0:5] = 0, 1, 1, 1, 1
    M[1,0:5] = 1, 0, A, B, C
    M[2,0:5] = 1, A, 0, D, F
    M[3,0:5] = 1, B, D, 0, E
    M[4,0:5] = 1, C, F, E, 0
    print(M) # comment out?
    return math.sqrt(np.linalg.det(M))/16  # Syn3 factored in 

The reference prime vector length is set to 1 in this formula, meaning the octahedron in which the icosahedron is inscribed, and what's left with the 24 S modules subtracted, is the diameter of a unit radius IVM sphere.

In [3]:
CM_ivm(EF, EG, EH, FG, GH, FH)

[[0.         1.         1.         1.         1.        ]
 [1.         0.         1.16718427 0.29179607 0.58359214]
 [1.         1.16718427 0.         0.8753882  1.52786405]
 [1.         0.29179607 0.8753882  0.         0.29179607]
 [1.         0.58359214 1.52786405 0.29179607 0.        ]]


0.045084971874737007

What's our usual value for the S mod volume?

In [4]:
φ         = (rt2(5)+1)/2   # golden ratio
Smod      = (φ**-5) / 2    # home base Smod
Smod

0.04508497187473711

Another Synergetics motif has been the S_Factor, or S_volume / E_volume...

In [5]:
Emod      = (rt2(2) / 8) * (φ**-3)  # home base Emod
S_factor  = Smod / Emod  # 2*sqrt(7-3*sqrt(5))

In [6]:
icosa_within = 2.5 * S_factor * S_factor
icosa_within + 24 * Smod

3.999999999999999

# Recomputing the S Module with sympy

That's the target Octahedron volume, or close.  The next step is to audit the above using `sympy` and its arbitrary precision numbers.

[sympy documentation](https://docs.sympy.org/latest/index.html)

In [7]:
from sympy import symbols, sqrt as rt2, Matrix, ones
import sympy as sy

In [8]:
a = symbols('a')

In [9]:
EF = a * rt2(7 - 3 * rt2(5))
EG = (a/2) * rt2(7 - 3 * rt2(5))
EH = (a/2) * (3 - rt2(5))
FG = (a/2) * rt2(3) * rt2(7 - 3 * rt2(5))
FH = (a/2) * (rt2(5) - 1)
GH = (a/2) * rt2(7 - 3 * rt2(5))

In [10]:
EF

a*sqrt(7 - 3*sqrt(5))

In [11]:
FH

a*(-1 + sqrt(5))/2

In [12]:
FH.evalf(50,subs={a: 1})  # to 50 decimal places

0.61803398874989484820458683436563811772030917980576

In [13]:
S_net = pd.DataFrame({'Edges'  : ['EF', 'EG', 'EH', 'FG', 'FH', 'GH'],
                      'Lengths': [EF, EG, EH, FG, FH, GH],
                      'Values' : [L.evalf(50, subs={a:1}) 
                                  for L in [EF, EG, EH, FG, FH, GH]]})

S_net.set_index('Edges', inplace=True)
S_net

,Lengths,Values
Edges,,
EF,a*sqrt(7 - 3*sqrt(5)),0.54018151347545290720308631409818785099473024...
EG,a*sqrt(7 - 3*sqrt(5))/2,0.27009075673772645360154315704909392549736512...
EH,a*(3 - sqrt(5))/2,0.38196601125010515179541316563436188227969082...
FG,sqrt(3)*a*sqrt(7 - 3*sqrt(5))/2,0.46781091332446829000553859902741320909259887...
FH,a*(-1 + sqrt(5))/2,0.61803398874989484820458683436563811772030917...
GH,a*sqrt(7 - 3*sqrt(5))/2,0.27009075673772645360154315704909392549736512...


In [14]:
S_net.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6 entries, EF to GH
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Lengths  6 non-null      object
 1   Values   6 non-null      object
dtypes: object(2)
memory usage: 144.0+ bytes


In [15]:
S_net.loc['EG', 'Values']

0.27009075673772645360154315704909392549736512170141

In [16]:
def CM_ivm(a, b, c, d, e, f):
    # double and 2nd power us
    A,B,C,D,E,F = [(2*x)**2 for x in (a,b,c,d,e,f)]
    
    # Construct a 5x5 matrix per Caley-Menger
    M = Matrix(
        [
        [0, 1, 1, 1, 1],
        [1, 0, A, B, C],
        [1, A, 0, D, F],
        [1, B, D, 0, E],
        [1, C, F, E, 0]])
    # print(M) # comment out?
    return rt2(M.det())/16  # Syn3 factored in

In [17]:
args = S_net.loc[['EF', 'EG', 'EH', 'FG', 'GH', 'FH'], 'Values']

In [18]:
args

Edges
EF    0.54018151347545290720308631409818785099473024...
EG    0.27009075673772645360154315704909392549736512...
EH    0.38196601125010515179541316563436188227969082...
FG    0.46781091332446829000553859902741320909259887...
GH    0.27009075673772645360154315704909392549736512...
FH    0.61803398874989484820458683436563811772030917...
Name: Values, dtype: object

In [19]:
Svol = CM_ivm(*args)
Svol

0.045084971874737120511467085914095294300772949514407

In [20]:
φ         = (rt2(5)+1)/2   # golden ratio
φ

1/2 + sqrt(5)/2

In [21]:
φ.evalf(50)

1.6180339887498948482045868343656381177203091798058

In [22]:
Svol2 = (φ**-5) / 2
Svol2

1/(2*(1/2 + sqrt(5)/2)**5)

In [23]:
Svol2 = Svol2.evalf(50)
Svol2

0.045084971874737120511467085914095294300772949514407

In [24]:
Svol

0.045084971874737120511467085914095294300772949514407

Looking good!

In [25]:
Evol      = (rt2(2) / 8) * (φ**-3)  # home base Emod
S_factor  = Svol / Evol  # 2*sqrt(7-3*sqrt(5))

In [26]:
2.5 * S_factor.evalf(50)**2

2.9179606750063091077247899380617129367814492116542

In [27]:
Svol

0.045084971874737120511467085914095294300772949514407

In [28]:
icosa_within = 2.5 * S_factor.evalf(50) * S_factor.evalf(50)
icosa_within + 24 * Svol

4.0000000000000000000000000000000000000000000000000

Yup!